In [2]:
import os

import optuna
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from lgbm_autotune.model.model_registry import ModelRegistry
from lgbm_autotune.model.objective import Objective
from lgbm_autotune.model.study_callback import StudyCallback

In [5]:
train_df = pd.read_csv("../lgbm_autotune/data/train.csv")
test_df = pd.read_csv("../lgbm_autotune/data/test.csv")

In [6]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked"]
label = "Survived"

for f in features:
    if (train_df[f].dtype == 'object'):
        le = LabelEncoder()
        train_df[f] = le.fit_transform(train_df[f])
        test_df[f] = le.transform(test_df[f])

X_train, X_test, y_train, y_test = train_test_split(
    train_df[features], train_df[label]
)

In [14]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,2
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,1
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,2
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,2


In [15]:
# initialize model registry and callback
model_registry = ModelRegistry("test")
callback = StudyCallback(model_registry)

# Your objective function should call model_registry.register() for each trial
objective = Objective(X_train, y_train, model_registry)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///optuna_study.db",
    study_name="test",
    load_if_exists=True,
)
study.optimize(objective, n_trials=10, callbacks=[callback])

best_model = model_registry.retrieve(study.best_trial.number)

y_pred = best_model.predict_proba(test_df[features])
print(y_pred)


[I 2023-06-18 17:20:29,885] A new study created in RDB with name: test
[I 2023-06-18 17:20:30,931] Trial 0 finished with value: 0.8216256454388985 and parameters: {'learning_rate': 0.032518983615173164, 'max_depth': 6, 'min_child_samples': 173, 'n_estimators': 1325, 'num_leaves': 16}. Best is trial 0 with value: 0.8216256454388985.
[I 2023-06-18 17:20:33,871] Trial 1 finished with value: 0.8511233505450374 and parameters: {'learning_rate': 0.14679282150449252, 'max_depth': 1, 'min_child_samples': 17, 'n_estimators': 5166, 'num_leaves': 209}. Best is trial 1 with value: 0.8511233505450374.
[I 2023-06-18 17:20:39,777] Trial 2 finished with value: 0.8489678714859439 and parameters: {'learning_rate': 0.03149647067194773, 'max_depth': 7, 'min_child_samples': 87, 'n_estimators': 5703, 'num_leaves': 212}. Best is trial 1 with value: 0.8511233505450374.
[I 2023-06-18 17:20:42,147] Trial 3 finished with value: 0.8310748709122203 and parameters: {'learning_rate': 0.07672573202972204, 'max_depth'

0.5


In [16]:
best_model = model_registry.retrieve(study.best_trial.number)

In [29]:
predictions = [model.predict(X_test) for model in best_model.models]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0])

In [11]:
best_trial = study.best_trial

print("Best trial:")
print(" Value: ", best_trial.value)
print(" Params: ", best_trial.params)
print(" All Params: ", best_trial.user_attrs["all_params"])

Best trial:
 Value:  0.8571718301778543
 Params:  {'learning_rate': 0.0734720086092719, 'max_depth': 5, 'min_child_samples': 85, 'n_estimators': 566, 'num_leaves': 179}
 All Params:  {'n_estimators': 566, 'num_leaves': 179, 'min_child_samples': 85, 'max_depth': 5, 'learning_rate': 0.0734720086092719, 'random_state': 123, 'boosting_type': 'gbdt', 'objective': 'binary'}


In [12]:
best_trial 

FrozenTrial(number=15, state=TrialState.COMPLETE, values=[0.8571718301778543], datetime_start=datetime.datetime(2023, 6, 18, 15, 56, 43, 397501), datetime_complete=datetime.datetime(2023, 6, 18, 15, 56, 44, 286293), params={'learning_rate': 0.0734720086092719, 'max_depth': 5, 'min_child_samples': 85, 'n_estimators': 566, 'num_leaves': 179}, user_attrs={'all_params': {'n_estimators': 566, 'num_leaves': 179, 'min_child_samples': 85, 'max_depth': 5, 'learning_rate': 0.0734720086092719, 'random_state': 123, 'boosting_type': 'gbdt', 'objective': 'binary'}}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=8, log=False, low=1, step=1), 'min_child_samples': IntDistribution(high=200, log=False, low=3, step=1), 'n_estimators': IntDistribution(high=10000, log=False, low=1, step=1), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1)}, trial_id=16, value=None)

In [15]:
bst = model_registry.load_best_model()

In [16]:
bst.models

[LGBMClassifier(learning_rate=0.0734720086092719, max_depth=5,
                min_child_samples=85, n_estimators=566, num_leaves=179,
                objective='binary', random_state=123),
 LGBMClassifier(learning_rate=0.0734720086092719, max_depth=5,
                min_child_samples=85, n_estimators=566, num_leaves=179,
                objective='binary', random_state=123),
 LGBMClassifier(learning_rate=0.0734720086092719, max_depth=5,
                min_child_samples=85, n_estimators=566, num_leaves=179,
                objective='binary', random_state=123),
 LGBMClassifier(learning_rate=0.0734720086092719, max_depth=5,
                min_child_samples=85, n_estimators=566, num_leaves=179,
                objective='binary', random_state=123),
 LGBMClassifier(learning_rate=0.0734720086092719, max_depth=5,
                min_child_samples=85, n_estimators=566, num_leaves=179,
                objective='binary', random_state=123)]

In [18]:
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_min_child_samples,params_n_estimators,params_num_leaves,user_attrs_all_params,state
0,0,0.839797,2023-06-18 15:54:18.459985,2023-06-18 15:54:19.841423,0 days 00:00:01.381438,0.179391,2,71,1644,172,"{'n_estimators': 1644, 'num_leaves': 172, 'min...",COMPLETE
1,1,0.727088,2023-06-18 15:54:19.882633,2023-06-18 15:54:24.823515,0 days 00:00:04.940882,0.125052,6,191,9022,113,"{'n_estimators': 9022, 'num_leaves': 113, 'min...",COMPLETE
2,2,0.852314,2023-06-18 15:54:24.838779,2023-06-18 15:54:30.465886,0 days 00:00:05.627107,0.016222,4,111,8077,3,"{'n_estimators': 8077, 'num_leaves': 3, 'min_c...",COMPLETE
3,3,0.825739,2023-06-18 15:54:30.684291,2023-06-18 15:54:35.689577,0 days 00:00:05.005286,0.159812,8,91,4902,90,"{'n_estimators': 4902, 'num_leaves': 90, 'min_...",COMPLETE
4,4,0.729499,2023-06-18 15:54:35.704840,2023-06-18 15:54:38.533929,0 days 00:00:02.829089,0.107966,6,191,5205,85,"{'n_estimators': 5205, 'num_leaves': 85, 'min_...",COMPLETE
5,5,0.832302,2023-06-18 15:54:38.549888,2023-06-18 15:54:44.049505,0 days 00:00:05.499617,0.188149,2,161,9147,103,"{'n_estimators': 9147, 'num_leaves': 103, 'min...",COMPLETE
6,6,0.828406,2023-06-18 15:54:44.064892,2023-06-18 15:54:47.754606,0 days 00:00:03.689714,0.246838,1,126,6818,130,"{'n_estimators': 6818, 'num_leaves': 130, 'min...",COMPLETE
7,7,0.834458,2023-06-18 15:54:47.770055,2023-06-18 15:54:57.768105,0 days 00:00:09.998050,0.121845,5,121,9494,138,"{'n_estimators': 9494, 'num_leaves': 138, 'min...",COMPLETE
8,8,0.817898,2023-06-18 15:54:57.783432,2023-06-18 15:55:11.375765,0 days 00:00:13.592333,0.021558,8,29,7135,79,"{'n_estimators': 7135, 'num_leaves': 79, 'min_...",COMPLETE
9,9,0.826764,2023-06-18 15:55:11.390235,2023-06-18 15:55:19.884153,0 days 00:00:08.493918,0.142263,8,108,9695,238,"{'n_estimators': 9695, 'num_leaves': 238, 'min...",COMPLETE
